In [37]:
import calcOneDay
import getDays
from datetime import datetime, timedelta
import calcTimeNow
import daysAndDates
import logging
import checkDST

def logdV3():
    
    """Set up logging for this script"""
    
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')
    #file_handler = logging.FileHandler('/home/ec2-user/davisPMUpdate.log')
    file_handler = logging.FileHandler('/Users/jameshayes/Sites/dV3Up.log')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)  

In [38]:
import getDays
from datetime import datetime, timedelta
import calcTimeNow
import daysAndDatesNew
import checkDST
from dataFromDavis import dataFromDavis
from getAndStore import getAndStore
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from dateutil.tz import tzutc, tzlocal
import pytz

def timesAndDates():
    
    """Calculate the time and date for end of day calculations
    First determine whether we are in DST or not"""

    # Check DST setting
    isDST = checkDST.checkDST()
    print(isDST)

    if isDST == 1:
        now = datetime.now()
        end = int(datetime.timestamp(now))
        start = (end - 59400)
        start = str(start)
        end = str(end)
        
    else:
        now = datetime.now()
        end = int(datetime.timestamp(now))
        start = (end - 59400)
        start = str(start)
        end = str(end)    

    dayInfo = daysAndDatesNew.daysAndDatesNew()
    month, month_num, date, year = dayInfo[0], dayInfo[1], dayInfo[2], dayInfo[3]
    yesterday = int(dayInfo[4])
    nextDay = int(dayInfo[5])
    month_num = int(month_num)
    date = int(date)
        
    return(start, end, month, month_num, date, year, yesterday, nextDay)  

In [39]:
import collections
import hashlib
import hmac
import time
from datetime import datetime
import requests
import json
import dataFile
import getData
import checkDST

#
# Get the start and end times, depending on product
#

def timeCheck():

    """make sure the time is correct for the product"""

    now = datetime.now()
    x = now.strftime("%Y-%m-%d")
    print("This script started at:", now)

    t1 = ' 17:00'
    t2 = ' 18:00'
    time1 = x + t1
    time2 = x + t2
    val1 = datetime.strptime(time1, "%Y-%m-%d %H:%M")
    val2 = datetime.strptime(time2, "%Y-%m-%d %H:%M")

    if val1 < now < val2:
        end = int(datetime.timestamp(now))
        start = (end - 59400)
        start = str(start)
        end = str(end)    
        
    else:
        pass

    return(start, end)
    

def dataFromDavis():
    
    """Get Davis data via the API""" 
       
    parameters = {
      "api-key": "vy8jbrjsxlbwgojepq3vfyfqfywyhvbd", 
      "api-secret": "sdqfm6wdfy9w0pqp2vdka38o6b4vcsvc",
      "station-id": 81211, 
      "end-timestamp": end,
      "start-timestamp": start,
      "t": int(time.time())
    }

    
    parameters = collections.OrderedDict(sorted(parameters.items()))

   #for key in parameters:
   #     print("Parameter name: \"{}\" has value \"{}\"".format(key, parameters[key]))

    apiSecret = parameters["api-secret"];
    parameters.pop("api-secret", None);

    data = ""
    for key in parameters:
        data = data + key + str(parameters[key])

    #logger.info('Data string to hash is: \"{}\"'.format(data))   


    apiSignature = hmac.new(
      apiSecret.encode('utf-8'),
      data.encode('utf-8'),
      hashlib.sha256
    ).hexdigest()

    #logger.info('API Signature is: \"{}\"'.format(apiSignature))

    # Building the URL to get the station
    
    first_part = ('https://api.weatherlink.com/v2/historic/81211?')
    api_key = ('api-key=vy8jbrjsxlbwgojepq3vfyfqfywyhvbd')
    add_apisig = ('&api-signature=')
    add_t = ('&t='+ str(int(time.time())))

    start1 = "&start-timestamp=" + start
    end1 = "&end-timestamp=" + end

    URLfinal = (first_part + api_key + add_t + start1 + end1 + add_apisig + apiSignature)

    r =  requests.get(URLfinal)
    davisAPI = (r.json())
    return(davisAPI)

dataFromDavis()

{'station_id_uuid': '245f6069-3309-4f78-aa0e-8f97d4e5d6ec',
 'sensors': [{'lsid': 274372,
   'data': [{'temp_in_lo_at': 1711084112,
     'arch_int': 60,
     'temp_in_hi': 73.5,
     'temp_in_hi_at': 1711084082,
     'hum_in_hi': 22.1,
     'temp_in_last': 73.3,
     'temp_in_lo': 73.3,
     'hum_in_lo': 22.1,
     'hum_in_last': 22.1,
     'dew_point_in': 32.3,
     'hum_in_lo_at': 1711084082,
     'tz_offset': -14400,
     'heat_index_in': 70.2,
     'hum_in_hi_at': 1711084082,
     'ts': 1711084140},
    {'temp_in_lo_at': 1711084172,
     'arch_int': 60,
     'temp_in_hi': 73.5,
     'temp_in_hi_at': 1711084142,
     'hum_in_hi': 22.3,
     'temp_in_last': 73.3,
     'temp_in_lo': 73.3,
     'hum_in_lo': 22.2,
     'hum_in_last': 22.2,
     'dew_point_in': 32.4,
     'hum_in_lo_at': 1711084172,
     'tz_offset': -14400,
     'heat_index_in': 70.2,
     'hum_in_hi_at': 1711084142,
     'ts': 1711084200},
    {'temp_in_lo_at': 1711084202,
     'arch_int': 60,
     'temp_in_hi': 73.4,


In [40]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import sqlalchemy
from dateutil.tz import tzutc, tzlocal
import pytz
import os
import dataFromDavis
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import math

#
# Create some F6 data from the API data and write to a Pandas dataFrame 
#

def getAndStore():
    
    davisAPI = dataFromDavis.dataFromDavis()
    a = davisAPI['sensors']    
    b = a[1]
    c = (b['data'])

    df = pd.DataFrame(c) 
    df.rename(columns = {'ts':'timestamp'}, inplace = True)
   
    timezone = pytz.timezone("America/New_York")
    df['timeGroup'] = pd.to_datetime(df['timestamp'], unit='s')
    df['timeGroup'] = df['timeGroup'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    df['localTime'] = df['timeGroup'].dt.strftime('%I:%M %p')

    df = df.loc[:,['timestamp', 'temp_hi', 'temp_hi_at','temp_lo', 'temp_lo_at', 'rainfall_in', 'dew_point_hi', 'dew_point_lo',  'rain_rate_hi_in', 'rain_rate_hi_at', 'timeGroup', 'localTime']]   
    
    print(df)
    return(df)

getAndStore()

#
# Calculate more variables from the dataFrame
#

import pandas as pd
from pandas import DataFrame, Series
import math

def createF6(df):
    max_temp  = (df.sort_values(by='temp_hi', ascending=False))
    max_T = max_temp.iloc[:1]
    max_T_time = int(max_T['temp_hi_at'])
    maxT = max_T['temp_hi'].values[0]
    maxT = round(maxT)

    min_temp  = (df.sort_values(by='temp_lo', ascending=True))
    min_T = min_temp.iloc[:1]
    min_T_time = int(min_T['temp_lo_at'])
    minT = min_T['temp_lo'].values[0]
    minT = round(minT)

    dew_max = (df.sort_values(by='dew_point_hi', ascending=False))
    dew_max1 = dew_max.iloc[:1]
    dewMax = dew_max1['dew_point_hi'].values[0]
    dewMaxT = round(dewMax)

    dew_min = (df.sort_values(by='dew_point_lo', ascending=True))
    dew_min1 = dew_min.iloc[:1]
    dewMin = dew_min1['dew_point_lo'].values[0]
    dewMinT = round(dewMin)

    rain = df['rainfall_in'].sum()
    avgTemp = math.ceil((int(maxT + minT)/2))

    hdd = (65 - avgTemp)
    if hdd < 0:
        hdd = 0
    cdd = (avgTemp - 65)
    if cdd < 0:
        cdd = 0         
     
    return(maxT, minT, dewMaxT, dewMinT, rain, avgTemp, hdd, cdd)

df = getAndStore()
createF6(df)

       timestamp  temp_hi  temp_hi_at  temp_lo  temp_lo_at  rainfall_in  \
0     1711056660     43.6  1711056614     43.6  1711056614            0   
1     1711056720     43.6  1711056677     43.6  1711056677            0   
2     1711056780     43.6  1711056740     43.6  1711056740            0   
3     1711056840     43.6  1711056792     43.6  1711056792            0   
4     1711056900     43.7  1711056887     43.6  1711056855            0   
...          ...      ...         ...      ...         ...          ...   
1435  1711142760     39.1  1711142717     39.0  1711142728            0   
1436  1711142820     39.1  1711142780     39.0  1711142812            0   
1437  1711142880     39.2  1711142875     39.1  1711142833            0   
1438  1711142940     39.1  1711142896     39.1  1711142896            0   
1439  1711143000     39.2  1711142959     39.1  1711142980            0   

      dew_point_hi  dew_point_lo  rain_rate_hi_in  rain_rate_hi_at  \
0             11.8          1

(44, 26, 28, 11, 0, 35, 30, 0)

In [41]:
import sandbox1
import sandbox2
import daysAndDatesNew
import createF6

#
# Import data for use in the alamanc
#

dayInfo = daysAndDatesNew.daysAndDatesNew()
month, month_num, date, year = dayInfo[0], dayInfo[1], dayInfo[2], dayInfo[3]
yesterday = int(dayInfo[4])
nextDay = int(dayInfo[5])

sdf = createF6.createF6(df)
print(sdf)
maxT, minT, dewMaxT, throttle, rain, avgTemp, hdd, cdd = sdf[0], sdf[1], sdf[2], sdf[3], sdf[4], sdf[5], sdf[6], sdf[7]

nmlData = sandbox2.sandbox2()
nmlHi = nmlData[3]
nmlLo = nmlData[4]

highData = sandbox1.recordHigh()
lowData = sandbox1.recordLow()
rainData = sandbox1.recordRain()

highPhrase = highData[2]
lowPhrase = lowData[2]
rainPhrase = rainData[2]

#filePath = '/var/www/html/000/climoDavisTest111.txt'
filePath = '/Users/jameshayes/Sites/climoDavis999.txt'

def almanac():
    
    """Set and and write the daily almanac"""

    with open(f'{filePath}','w') as outfile1: 
    #with open('/var/www/html/000/climoDavisText.txt','w') as outfile1: 
        print(f'Daily almanac for {month} {date}, {year}', file = outfile1)
        print('\n', file = outfile1)
        print(f'The high so far today is {maxT} degrees', file = outfile1)
        print(f'The low so far today is {minT} degrees', file = outfile1)
        print(f'The average temperature is {avgTemp} degrees', file = outfile1)
        print(f'The rainfall so far today is {("%.2f" % rain)} inches', file = outfile1)
        if hdd == 0:
            print('')
        else:
            print(f'There were {hdd} heating degree days', file = outfile1)
        if cdd == 0:
            print('')
        else:
            print(f'There were {cdd} cooling degree days', file = outfile1)
            
        print('\n', file = outfile1)        
    
    
        if date == 1:
            print(f'Normal and Record information for {nextMonth} {nextDay}, {year}', file = outfile1)
            print('\n', file = outfile1)
            print(f'The normal high for today is {nmlHi} degrees', file = outfile1)
            print(f'The normal low for today is {nmlLo} degrees' , file = outfile1)
            print('\n', file = outfile1)
            print(highPhrase, file = outfile1)
            print(lowPhrase, file = outfile1)
            print(rainPhrase, file = outfile1)  
        
        else:
            print(f'Normal and Record information for {month} {date}, {year}', file = outfile1)
            print('\n', file = outfile1)
            print(f'The normal high for today is {nmlHi} degrees', file = outfile1)
            print(f'The normal low for today is {nmlLo} degrees' , file = outfile1)
            print('\n', file = outfile1)
            print(highPhrase, file = outfile1)
            print(lowPhrase, file = outfile1)
            print(rainPhrase, file = outfile1)      

('March', '03', '21', '2024')
(44, 26, 28, 11, 0, 35, 30, 0)
1
2003
